# Generate Sandpiles
This notebook serves to simulate sandpiles using the sandpile model. The resulting avalanches with all relevant physical quantities are saved as `.feather` files for fast writing and reading and reduced storage usage.

To save you the time of generating avalanches, our generated avalanches are available in the corresponding folders (`avalanches*/`).

In [ ]:
from dataclasses import dataclass
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import os

# Pretty matplotlib plots
plt.rcParams.update(
    {
        "figure.dpi": 300,
        "font.family": "serif",
        "font.serif": "cm",
        "mathtext.fontset": "cm",
        # INFO: Enable this, in case you want latex rendering
        # "text.usetex": True,
        "axes.grid": True,
        "grid.linewidth": 0.2,
        "grid.alpha": 0.5,
    }
)

@dataclass
class AvalancheData:
    """A dataclass that holds all relevant observables of an avalanche."""
    lifetime: int
    size: int
    linear_size: float
    origin: tuple[int, ...]
    topple_times: np.ndarray
    z_avg: float
    corr:float


def change_neighbors(z, r, direction, delta):
    """Increases all neighbors of the size r in the given direction (+1 or -1) by delta."""
    d = z.ndim
    N = z.shape[0]

    # increase neighbors
    for i in range(d):
        r_neighbor = np.array(list(r))
        r_neighbor[i] += direction

        if r_neighbor[i] < 0 or r_neighbor[i] >= N:
            continue

        z[tuple(r_neighbor)] += delta

def avalanche_step(z: np.ndarray, data: AvalancheData, closed=False):
    """
    Simulates a single step of an avalanche, i.e. a single relaxation, on the slope lattice z.

    The observables are stored in the data object.
    Closed boundary conditions are used if closed is True.
    Returns True if any sites were relaxed, False otherwise.
    """
    d = z.ndim
    N = z.shape[0]
    z_c = 2 * d - 1
    shape_arr = np.array(z.shape)
    
    # impose boundary conditions
    for i in range(d):
        # set all sites on the 0-boundary to 0
        z[tuple(
            [0] * i
            + [slice(None)]
            + [0] * (d - i - 1)
        )] = 0

        if closed:
            # set all sites on the (N - 1)-boundary to 0
            z[tuple(
                [N - 1] * i
                + [slice(None)]
                + [N - 1] * (d - i - 1)
            )] = 0

    # find unstable sites
    unstable_sites = np.argwhere(z > z_c)
    # compute instantaneous dissipation rate
    f_alpha = len(unstable_sites)

    # if there are no unstable sites, the avalanche is over
    if f_alpha == 0:
        return False

    data.lifetime += 1

    # compute total dissipation rate (size) by summing up
    # the instantaneous dissipation rates
    data.size += f_alpha
    
    for r in unstable_sites:
        # If the distance to the origin site is larger than the linear_size,
        # then update the linear_size.
        distance_to_origin = np.linalg.norm(r - data.origin)
        if distance_to_origin > data.linear_size:
            data.linear_size = distance_to_origin

        r_tup = tuple(r)

        border_component_count = np.sum(r == shape_arr - 1)
        
        # relax site
        z[r_tup] -= 2 * d - border_component_count
        if data.topple_times is not None:
            data.topple_times[r_tup] = data.lifetime

        # increase neighbors in both directions
        change_neighbors(z, r, -1, delta=1)
        change_neighbors(z, r, 1, delta=1)

    # return True, since we relaxed at least one site
    return True


def create_avalanche_data(z, origin, include_topple_times=False):
    """Helper function to create an initial AvalancheData object."""

    return AvalancheData(
        lifetime=0,
        size=0,
        linear_size=0,
        origin=origin,
        topple_times=np.full(z.shape, -1, dtype=np.int16) if include_topple_times else None,
        z_avg=None,
        corr = None
    )


def start_avalanche(z: np.ndarray, origin: tuple, closed=False, include_topple_times=False):
    """
    Starts an avalanche on the slope lattice z at the given origin site.
    
    The avalnche will be fully relaxed until the lattice is stable.
    Returns an AvalancheData object that holds all relevant observables.
    """

    d = z.ndim
    z_c = 2 * d - 1

    assert np.sum(z > z_c) <= 1, "there cannot be more than one unstable site"

    data = create_avalanche_data(z, origin, include_topple_times)
    
    did_change = True

    while did_change: 
        did_change = avalanche_step(z, data, closed=closed)

    assert np.all(z <= z_c), "z must be stable after avalanche"

    data.z_avg = np.mean(z)
    
    return data


def evolve(z, steps=None, progress=True, closed=False, cons=False, include_topple_times=False):
    """
    Evolves the slope lattice z for the given number of steps.
    
    In each step, a random site is chosen and perturbed conservatively if cons=True or
    non-conservatively otherwise.
    Returns a list of AvalancheData objects that hold all relevant observables.
    """

    d = z.ndim
    N = z.shape[0]
    z_c = 2 * d - 1
    if np.any(z > z_c):
        raise ValueError("z must be stable before evolving")

    if steps is None:
        therm_steps_approx = z_c * N**d
        steps = therm_steps_approx + 100000

    avalanches = []

    for step in tqdm(range(steps), disable=not progress):
        # print(f"STEP {step}")
        r = tuple([np.random.randint(0, s) for s in z.shape])

        if cons:
            z[r] += d
            change_neighbors(z, r, -1, delta=-1)
        else:
            z[r] += 1

        avalanche_data = start_avalanche(z, r, closed=closed, include_topple_times=include_topple_times)
        avalanches.append(avalanche_data)

    return avalanches

def create_zero_z(d, N, cons=False):
    """
    Creates an initial slope lattice z with all sites set to 0.
    
    The cons argument determines whether negative slopes are allowed or not.
    """

    return np.zeros((N,) * d, dtype=np.int16 if cons else np.uint16)

In [ ]:
def generate_avalanches(d, N, progress=True, closed=False, cons=False):
    """
    Generates avalanches on a slope lattice of dimension d and size N and
    stores them as a DataFrame in a feather file.
    """

    z = create_zero_z(d, N, cons=cons)
    avalanches = evolve(z, progress=progress, closed=closed, cons=cons)

    # Create DataFrame from list of AvalancheData objects.
    # This will create a column for each attribute of AvalancheData.
    df = pd.DataFrame(avalanches)

    # Remove topple times, since they are not needed for the analysis
    df.drop(columns=["topple_times"], inplace=True)

    # Choose the correct directory for the avalanches
    if cons and closed:
        dir_path = f"avalanches_cons_closed/d{d}"
    elif cons:
        dir_path = f"avalanches_cons/d{d}"
    elif closed:
        dir_path = f"avalanches_closed/d{d}"
    else:
        dir_path = f"avalanches/d{d}"

    # Save avalanches
    os.makedirs(dir_path, exist_ok=True)
    df.to_feather(f"{dir_path}/N{N}.feather")

## Comparison of boundary conditions and perturbation methods
Here we generate avalanches for different boundary conditions and perturbation methods which can be compared later.

In [ ]:
# some avalanches (open boundary, non-conservative)
for d, N in [(2, 40), (3, 20), (4, 10)]:
    generate_avalanches(d=d, N=N)

In [ ]:
# some avalanches with closed boundaries, to compare with open
for d, N in [(2, 40), (3, 20), (4, 10)]:
    generate_avalanches(d=d, N=N, closed=True)

In [ ]:
# some avalanches with conservative perturbation, to compare with non-conservative
for d, N in [(2, 40), (3, 20), (4, 10)]:
    generate_avalanches(d=d, N=N, cons=True)

In [ ]:
# some avalanches with conservative perturbation and closed boundaries
for d, N in [(2, 40), (3, 20), (4, 10)]:
    generate_avalanches(d=d, N=N, cons=True, closed=True)

## Base data for other analyses
(Note: These cells will take several hours to execute.)

In [ ]:
# d = 2

for N in np.arange(4, 41, 1):
    print(f"N = {N}")
    generate_avalanches(d=2, N=N)

In [ ]:
# d = 3

for N in np.arange(4, 31, 1):
    print(f"N = {N}")
    generate_avalanches(d=3, N=N)

In [ ]:
# d = 4

for N in np.arange(14, 21, 1):
    print(f"N = {N}")
    generate_avalanches(d=4, N=N)

## Avalanche visualization
Now, we visualize the temporal evolution of several avalanches.

In [ ]:
from matplotlib.colors import ListedColormap

def plot_avalanches(df, ncols=6, filename="avalanches.png"):
    """
    Plots the temporal evolution of the avalanches in the given DataFrame.
    """

    # determine number of rows for the plot
    nrows = int(np.ceil(len(df) / ncols))

    fig, axes = plt.subplots(nrows, ncols, figsize=(ncols * 2, nrows * 2), squeeze=False, dpi=300)
    axes = axes.flatten()

    # hide plot axes
    for ax in axes:
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

    # determine max lifetime for colorbar
    max_lifetime = max(av.lifetime for _, av in df.iterrows())

    # create color map with white for untoppled sites
    cmap = plt.get_cmap("viridis")
    colors = cmap(np.linspace(0, 1, max_lifetime + 1))
    colors[0] = (1, 1, 1, 1)
    cmap = ListedColormap(colors)

    for i, avalanche in df.iterrows():
        # show topple times of different sites as colored pixels
        im = axes[i].imshow(avalanche.topple_times, vmin=-1, vmax=max_lifetime, cmap=cmap)

        # show relevant variables of avalanche
        axes[i].text(1, 1,
            (
                f"$t = {avalanche.lifetime:.0f}$\n"
                f"$s = {avalanche.size:.0f}$\n"
                f"$l = {avalanche.linear_size:.1f}$"
            ),
            color="k", va="top", ha="left", fontsize=9,
            bbox=dict(facecolor="w", alpha=.8, edgecolor="none")
        )

        # show origin of avalanche
        axes[i].plot(avalanche.origin[1], avalanche.origin[0], "x", c="orange", ms=8, mew=1.5)

    # add colorbar
    colorbar_ax = fig.add_axes([.92, .2, .01, .6])
    fig.colorbar(im, cax=colorbar_ax, label="topple time")
        
    plt.subplots_adjust(wspace=.05, hspace=.05)
    plt.savefig(f"Report/figs/{filename}", bbox_inches="tight")

    return fig, axes


def find_avalanches(z, count, condition):
    """
    Evolves the given slope lattice z until the specified number of avalanches
    which fulfill the given condition is found.
    """
    avalanches = []

    while len(avalanches) < count:
        data = evolve(z, steps=1, progress=False, include_topple_times=True)[0]

        if condition(data):
            avalanches.append(data)

    return avalanches


# Visualize some avalanches with high lifetime
z = create_zero_z(2, 40)
some_long_avalanches = find_avalanches(z, 24, lambda av: av.lifetime >= 100)

plot_avalanches(some_long_avalanches);